# OCR pour Caractères Manuscrits Chinois avec OpenVINO™

Dans ce tutoriel, nous réalisons la reconnaissance optique de caractères (OCR) pour des caractères manuscrits en chinois simplifié. Un guide OCR pour l’alphabet latin est également disponible dans le [notebook 208](../optical-character-recognition/optical-character-recognition.ipynb). Le modèle utilisé ici est conçu pour traiter une seule ligne de caractères à la fois.

Le modèle employé dans ce notebook est [`handwritten-simplified-chinese-0001`](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/handwritten-simplified-chinese-recognition-0001/README.md). Les sorties du modèle sont décodées en texte lisible grâce aux listes de caractères [`scut_ept`](https://github.com/openvinotoolkit/open_model_zoo/blob/master/data/dataset_classes/scut_ept.txt). Ces modèles sont disponibles dans l’[Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/).

La configuration de ce modèle est inspirée du guide suivant : [Guide de configuration OCR](https://docs.openvino.ai/2024/notebooks/handwritten-ocr-with-output.html).

### Guide d'Installation

Ce tutoriel fonctionne de manière autonome, reposant uniquement sur son propre code.

Il est recommandé d’exécuter ce notebook dans un environnement virtuel. Seul un serveur Jupyter est nécessaire pour démarrer. Pour davantage de détails, consultez le [Guide d'Installation](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/handwritten-ocr/handwritten-ocr.ipynb" />

## Import de modules

In [ ]:
# Installer les paquets nécessaires
%pip install -q "openvino>=2023.1.0" opencv-python tqdm "matplotlib>=3.4"

from collections import namedtuple, defaultdict
from itertools import groupby
import cv2
import matplotlib.pyplot as plt
import numpy as np
import openvino as ov
import requests
from pathlib import Path

# Récupérer le module `notebook_utils`
r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)
open("notebook_utils.py", "w").write(r.text)
from notebook_utils import download_file, device_widget
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
!pip install -U jupyter ipywidgets notebook

## Téléchargement et Configuration du modèle

In [ ]:
# Définition des répertoires pour les modèles et les données
base_models_dir = "./chinese_handwritten_models"  # Répertoire pour les modèles téléchargés
data_folder = "./chinese_handwritten_data"  # Répertoire pour les données d'entrée
charlist_folder = f"{data_folder}/text"  # Sous-répertoire pour la liste de caractères

# Spécification de la précision du modèle (FP16 pour une inférence rapide et légère)
precision = "FP16"

# Structure nommée 'Language' pour regrouper les informations du modèle
Language = namedtuple("Language", ["model_name", "charlist_name", "demo_image_name"])

# Initialisation des fichiers pour le chinois simplifié
chinese_files = Language(
    model_name="handwritten-simplified-chinese-recognition-0001",
    charlist_name="chinese_charlist.txt",
    demo_image_name="handwritten_chinese_test.jpg",
)

# Téléchargement des fichiers du modèle OpenVINO
path_to_model = download_file(
    url=f"https://storage.openvinotoolkit.org/repositories/open_model_zoo/2023.0/models_bin/1/{chinese_files.model_name}/{precision}/{chinese_files.model_name}.xml",
    directory=base_models_dir,
)

# Téléchargement du fichier .bin contenant les poids du modèle
_ = download_file(
    url=f"https://storage.openvinotoolkit.org/repositories/open_model_zoo/2023.0/models_bin/1/{chinese_files.model_name}/{precision}/{chinese_files.model_name}.bin",
    directory=base_models_dir,
)

# Initialisation d'OpenVINO pour l'inférence
core = ov.Core()

# Chargement du modèle avec son fichier XML
model = core.read_model(model=path_to_model)

# Sélection du périphérique pour l'inférence avec un widget interactif
device = device_widget()  # Assurez-vous que device_widget est défini

# Compilation du modèle pour le dispositif sélectionné
compiled_model = core.compile_model(model=model, device_name=device.value)

# Extraction des couches d'entrée et de sortie du modèle
recognition_output_layer = compiled_model.output(0)  # Couche de sortie
recognition_input_layer = compiled_model.input(0)  # Couche d'entrée

# Téléchargement de la liste de caractères pour le décodage des prédictions
used_charlist_file = download_file(
    f"https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/text/{chinese_files.charlist_name}",
    directory=charlist_folder,
)

# Préparation de la liste de caractères pour le décodage
blank_char = "~"
with open(used_charlist_file, mode="r", encoding="utf-8") as charlist:
    letters = blank_char + "".join(line.strip() for line in charlist)  # Ajout d'un caractère vide

## Fonction de prédiction

In [ ]:
def OCR_OpenVINO(compiled_model, input_directory, charlist, recognition_input_layer, recognition_output_layer, reference_from) -> list:
    """
    Effectue la reconnaissance optique de caractères (OCR) sur des images d'un répertoire donné 
    à l'aide d'un modèle OpenVINO.

    :param compiled_model: Modèle compilé d'OpenVINO pour l'inférence.
    :param input_directory: Répertoire contenant les images à traiter.
    :param charlist: Liste des caractères utilisée pour le décodage (incluant un symbole vide).
    :param recognition_input_layer: Couche d'entrée du modèle de reconnaissance.
    :param recognition_output_layer: Couche de sortie du modèle de reconnaissance.
    :param reference_from: Indicateur pour définir la source des références ("file_name" ou "dir_name").
    :return: Liste des textes prédits pour chaque image ainsi que les références associées.
    """
    # Liste pour stocker les textes prédits pour chaque image
    predictions = []
    references = []

    # Parcourir chaque image dans le répertoire spécifié
    for path_image in Path(input_directory).glob("*"):
        if path_image.suffix in (".jpg", ".png", ".jpeg"):  # Vérifier l'extension de fichier
            # Lire l'image en niveaux de gris
            image = cv2.imread(str(path_image), cv2.IMREAD_GRAYSCALE)
    
            if image is None:
                print(f"Avertissement : Impossible de lire l'image {path_image}. Passage à l'image suivante.")
                continue
    
            # Obtenir les dimensions de l'image
            image_height, image_width = image.shape
    
            # Récupérer la forme d'entrée du modèle
            _, _, H, W = recognition_input_layer.shape
    
            # Calculer le ratio de redimensionnement pour ajuster la hauteur
            scale_ratio = H / image_height
    
            # Redimensionner l'image en fonction de la hauteur
            new_width = int(image_width * scale_ratio)
            resized_image = cv2.resize(image, (new_width, H), interpolation=cv2.INTER_AREA)
    
            # Compléter l'image pour correspondre à la largeur d'entrée sans changer l'aspect
            if resized_image.shape[1] < W:
                resized_image = np.pad(resized_image, ((0, 0), (0, W - resized_image.shape[1])), mode="constant", constant_values=255)
    
            # Reshape pour correspondre à la forme d'entrée du modèle
            input_image = resized_image[None, None, :, :]  # Préparation pour l'entrée réseau
    
            # Effectuer l'inférence avec le modèle
            predictions_output = compiled_model([input_image])[recognition_output_layer]
    
            # Supprimer la dimension de lot
            predictions_output = np.squeeze(predictions_output)
    
            # Obtenir les indices des classes prédites
            predictions_indexes = np.argmax(predictions_output, axis=1)
    
            # Utiliser `groupby` pour supprimer les lettres consécutives selon le décodage CTC
            output_text_indexes = list(groupby(predictions_indexes))
            output_text_indexes, _ = np.transpose(output_text_indexes, (1, 0))  # Éliminer les objets grouper
            output_text_indexes = output_text_indexes[output_text_indexes != 0]  # Supprimer les symboles vides
            
            # Mapper les indices aux caractères correspondants
            output_text = "".join([charlist[letter_index] for letter_index in output_text_indexes])
    
            # Stocker la prédiction
            predictions.append(output_text if output_text else "None")

            # Récupérer la référence selon le type spécifié
            if reference_from == "file_name":
                references.append(path_image.stem[0])  # Prendre le premier caractère du nom de fichier
            elif reference_from == "dir_name":
                references.append(str(path_image.parent.name))  # Prendre le nom du répertoire parent

    return predictions, references

## Fonction d'évaluation

In [ ]:
def evaluation(predictions, references):
    """
    Évalue la précision, le rappel et le score F1 pour une tâche OCR
    avec des images correspondant à un seul caractère. 
    Retourne les moyennes macro et weighted.

    :param predictions: Liste des caractères prédits par le modèle.
    :param references: Liste des caractères de référence (attendus).
    :return: Dictionnaire des scores de précision, rappel et F1 pour les moyennes macro et weighted,
             ainsi qu'un rapport de classification détaillé pour chaque caractère de référence.
    """
    if not predictions:  # Vérifie si la liste des prédictions est vide
        print("Erreur : Les prédictions sont vides.")
        return {
            "Précision (macro)": 0, "Rappel (macro)": 0, "Score F1 (macro)": 0,
            "Précision (weighted)": 0, "Rappel (weighted)": 0, "Score F1 (weighted)": 0
        }, {}

    # Calcul des scores avec gestion des divisions par zéro
    results = {
        "Précision (macro)": precision_score(references, predictions, average="macro", zero_division=0),
        "Rappel (macro)": recall_score(references, predictions, average="macro", zero_division=0),
        "Score F1 (macro)": f1_score(references, predictions, average="macro", zero_division=0),
        "Précision (weighted)": precision_score(references, predictions, average="weighted", zero_division=0),
        "Rappel (weighted)": recall_score(references, predictions, average="weighted", zero_division=0),
        "Score F1 (weighted)": f1_score(references, predictions, average="weighted", zero_division=0)
    }

    # Crée un rapport détaillé en utilisant uniquement les labels présents dans les références
    unique_labels = sorted(set(references))
    classification_rep = classification_report(references, predictions, labels=unique_labels, output_dict=True, zero_division=0)

    return results, classification_rep

In [ ]:
# Caractères Guilhem
test_directory = "Char/"
all_predictions_chars_Guilhem, all_references_chars_Guilhem = OCR_OpenVINO(compiled_model,
                                                                           test_directory,
                                                                           letters,
                                                                           recognition_input_layer,
                                                                           recognition_output_layer,
                                                                           reference_from="file_name")

In [ ]:
# Caractères Corpus Test
test_directory = "data/NewData120/Test/"
all_predictions_120Test, all_references_120Test = [], []
for directory in Path(test_directory).iterdir():
    predictions_per_char, references_per_chars = OCR_OpenVINO(compiled_model,
                                                              directory,
                                                              letters,
                                                              recognition_input_layer,
                                                              recognition_output_layer,
                                                              reference_from="dir_name")
    all_predictions_120Test += predictions_per_char
    all_references_120Test += references_per_chars

In [ ]:
import pickle

# Chemin de sauvegarde pour les fichiers pickle
predictions_path = "all_predictions_120Test.pkl"
references_path = "all_references_120Test.pkl"

# Sauvegarder les prédictions
with open(predictions_path, "wb") as f:
    pickle.dump(all_predictions_120Test, f)

# Sauvegarder les références
with open(references_path, "wb") as f:
    pickle.dump(all_references_120Test, f)

In [ ]:
# Évaluation
results_120Test, report_120Test = evaluation(all_predictions_120Test, all_references_120Test)
results_chars_Guilhem, report_chars_Guilhem = evaluation(all_predictions_chars_Guilhem, all_references_chars_Guilhem)

print("Scores globaux pour le corpus de test de Guilhem :")
for metric, score in results_chars_Guilhem.items():
    print(f"{metric}: {score:.4f}")
print()

print("Scores globaux pour le corpus de test  :")
for metric, score in results_120Test.items():
    print(f"{metric}: {score:.4f}")


print("\nRapport de classification détaillé pour le corpus de test :")
for label, metrics in report_120Test.items():
    if label not in ['micro avg', 'macro avg', 'weighted avg']:
        print(f"Caractère : {label}")
    else:
        print(label)
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()